Web scraping indeed website for data science jobs based on Greg Reda's [excellent tutorial](http://www.gregreda.com/2013/03/03/web-scraping-101-with-python/) and Jesse Steinweg's [excellent analysis](https://jessesw.com/Data-Science-Skills/) and finally, Sung Pil Moon's [awesome analysis](http://blog.nycdatascience.com/students-work/project-3-web-scraping-company-data-from-indeed-com-and-dice-com/)

## 1) Admin and Setup

In [1]:
from bs4 import BeautifulSoup
from urllib2 import urlopen
import pandas as pd
import re
import numpy as np
from time import sleep
%matplotlib inline
import pylab as plt
import os

## 2) Create a function for data collection

In [2]:
def get_data_job_board(job_query1):
    #STEP1
    # create query string in the required format
    # 1) within search string, spaces are replaced by '+' in html
    # 2) each search query is preceded and succeeded by a "%22"
    # 3) string the elements of the list into one string separated by a "+"
    job_query_string = []
    job_query_string.append(job_query1.replace(" ","+") + "&l=")
    #job_query_string.append("%22" + job_query2.replace(" ","+") + "%22")
    #job_query_string.append("%22" + job_query3.replace(" ","+") + "%22")
    job_query_string = "+".join(job_query_string)
    job_query = job_query_string.replace("+"," ")
    job_query = job_query.replace("&l=","")
    
    stepname = 'one'
    
    #STEP2
    # start of the url - this will not change because I'm including search query to search
    # anywhere in the job ad. Not just the job title.
    base_url = 'http://au.indeed.com/jobs?q='
    
    stepname = 'two'
    
    #STEP3
    #company name, salary, location and fromage are not being worked on currently
    #pagenum is null for the first time the query is generated
    company_name_string=''
    salary_string=''
    location_string=''
    fromage_string='any'
    
    stepname = 'three'
    
    #STEP4
    #create query to read the page for the first time
    initial_query = [base_url,job_query_string,'&as_not=&as_ttl=&as_cmp=',company_name_string,
                '&jt=all&st=&salary=',salary_string,'&radius=50&l=',location_string,
                '&fromage=',fromage_string,'&limit=10&sort=&']
    initial_query = "".join(initial_query)
    
    stepname = 'four'
    
    #open website and read it
    try:
        html = urlopen(initial_query).read()  
        soup = BeautifulSoup(html, "lxml")
        
        stepname = 'five'
    
        #find how many times to loop through to get each job
        number_of_jobs_page_area = soup.find(id="searchCount").string.encode('utf-8')
        number_of_jobs = re.findall('\d+', number_of_jobs_page_area.replace(',',''))
        total_number_of_jobs = int(number_of_jobs[2])
        number_of_pages_to_scroll = np.ceil(total_number_of_jobs/10)
        
        print total_number_of_jobs,' jobs found for search terms:',job_query1.upper()
        
        if total_number_of_jobs > 1000: #realised that job board has limitation to show only 1000 records
            print 'Limiting Search to first 1000 records...'
        
        stepname = 'six'
            
        #initialise variables for storing page variables
        jobtitle = []
        companyname = []
        location = []
        advertised_number_of_days_ago = []
        company_rating = []
        company_rating_counts = []
        salary = []
        summary = []
        joblink = []
        previous_job_number_start = '' # this will be used to determine when to break loop 
                                       # (indeed repeats job postings)       
        
        #loop through
        for counter in xrange(0,100):
         
            stepname = 'seven'
            
            #create query to read the page
            sleep(0.1)   #giving it a one second buffer because we don't want to overload server
            page_query = [base_url,job_query_string,'&as_not=&as_ttl=&as_cmp=',company_name_string,
                   '&jt=all&st=&salary=',salary_string,'&radius=50&l=',location_string,
                   '&fromage=',fromage_string,'&limit=10&sort=&start=',str(counter * 10)]
            page_query = "".join(page_query)
            
            stepname = 'eight'
            
            #read page contents
            html = urlopen(page_query).read()  
            soup = BeautifulSoup(html, "lxml")
            
            #print page numbers for each page fetch
            current_page_number_of_jobs_page_area = soup.find(id="searchCount").string.encode('utf-8')
            current_page_number_of_jobs = re.findall('\d+', current_page_number_of_jobs_page_area.replace(',',''))
            
            #this test is to not repeat job fetches. indeed removes the last couple of hundred job postings
            #because they are similar to what has been previously displayed. We remove it from our fetch too
            if int(current_page_number_of_jobs[0]) == previous_job_number_start:
                break
            previous_job_number_start = int(current_page_number_of_jobs[0])
            
            print '       fetching details for jobs', int(current_page_number_of_jobs[0]), \
            'to',int(current_page_number_of_jobs[1])
            
            #extract page body / details into a BS4 element result set called targetElements
            targetElements = soup.findAll('div', attrs = {'class' : ' row result'})
            targetElements.extend(soup.findAll('div', attrs = {'class' : 'lastRow row result'}))
            
            #start storing into variable lists
            for elem in targetElements:
                
                #job title
                jobtitle.append(elem.find('a', attrs = {'class':'turnstileLink'}).attrs['title'])              
                
                #company name
                if elem.find('span', attrs = {'itemprop':'name'}) is None:
                    companyname.append(None)
                else:
                    companyname.append(elem.find('span', attrs = {'itemprop':'name'})
                                     .getText().strip().encode('utf-8'))  
                
                #location
                location.append(elem.find('span', attrs = {'itemprop':'addressLocality'})
                                .getText().strip().encode('utf-8'))
                
                #summary
                summary.append(elem.find('span', attrs = {'class':'summary'})
                               .getText().strip().encode('utf-8'))
                
                #company rating
                if elem.find('span', attrs = [{'class':'ratingNumber'}]) is None:
                    company_rating.append(None)
                else:
                    company_rating.append(elem.find('span', attrs = {'class':'ratingNumber'})
                                     .getText().strip().encode('utf-8'))
                
                #company rating counts
                if elem.find('span', attrs = {'class':'slNoUnderline'}) is None:
                    company_rating_counts.append(None)
                else:
                    company_rating_counts.append(elem.find('span', attrs = {'class':'slNoUnderline'})
                                     .getText().strip().encode('utf-8'))
                
                #advertised number of days ago
                if elem.find('span', attrs = {'class':'date'}) is None:
                    advertised_number_of_days_ago.append(None)
                else:
                    advertised_number_of_days_ago.append(elem.find('span', attrs = {'class':'date'})
                                     .getText().strip().encode('utf-8'))
                    
                #salary
                if elem.find('nobr') is None:
                    salary.append(None)
                else:
                    salary.append(elem.find('nobr').getText().strip().encode('utf-8'))
                
                #job link
                home_url = 'http://www.indeed.com'
                joblink.append("%s%s" % (home_url,elem.find('a').get('href')))
                
        #after all pages are extracted, store them into a dataframe
        df_columns=['query_date','jobtitle','companyname','location',
                 'advertised_number_of_days_ago','company_rating',
                 'company_rating_counts','salary','summary',
                 'joblink','job_query']

        df_joblist = pd.DataFrame({'query_date':pd.to_datetime('today'),
                                'jobtitle':jobtitle,
                                'companyname':companyname,
                                'location':location,
                                'advertised_number_of_days_ago':advertised_number_of_days_ago,
                                'company_rating':company_rating,
                                'company_rating_counts':company_rating_counts,
                                'salary':salary,
                                'summary':summary,
                                'joblink':joblink,
                                'job_query':job_query},
                                 columns = df_columns)
            
        return df_joblist    

    except:
        print '0   jobs found for search term: ' + job_query.upper()
        #df_columns=['query_date','jobtitle','companyname','location',
        #        'advertised_number_of_days_ago','company_rating',
        #         'company_rating_counts','salary','summary',
        #         'joblink','job_query']
        #
        #df_joblist = pd.DataFrame({'query_date':pd.to_datetime('today'),
        #                        'jobtitle':jobtitle,
        #                        'companyname':companyname,
        #                        'location':location,
        #                        'advertised_number_of_days_ago':advertised_number_of_days_ago,
        #                        'company_rating':company_rating,
        #                        'company_rating_counts':company_rating_counts,
        #                        'salary':salary,
        #                        'summary':summary,
        #                        'joblink':joblink,
        #                        'job_query':job_query_string},
        #                         columns = df_columns)        
        #return df_joblist

## 3) Query and Download

In [3]:
jobs = pd.read_csv('jobs.csv', encoding='utf-8')
jobs = jobs["Jobs"]
df_joblist = pd.DataFrame()
for job in jobs:
    df_job = get_data_job_board(job)
    df_joblist = pd.concat((df_joblist,df_job))
df_joblist.reset_index(inplace = True)
del df_joblist["index"]

1993  jobs found for search terms: CUSTOMER RELATIONS
Limiting Search to first 1000 records...
       fetching details for jobs 1 to 10
       fetching details for jobs 11 to 20
       fetching details for jobs 21 to 30
       fetching details for jobs 31 to 40
       fetching details for jobs 41 to 50
       fetching details for jobs 51 to 60
       fetching details for jobs 61 to 70
       fetching details for jobs 71 to 80
       fetching details for jobs 81 to 90
       fetching details for jobs 91 to 100
       fetching details for jobs 101 to 110
       fetching details for jobs 111 to 120
       fetching details for jobs 121 to 130
       fetching details for jobs 131 to 140
       fetching details for jobs 141 to 150
       fetching details for jobs 151 to 160
       fetching details for jobs 161 to 170
       fetching details for jobs 171 to 180
       fetching details for jobs 181 to 190
       fetching details for jobs 191 to 200
       fetching details for jobs 201 to 210
 

In [4]:
def compute_advertised_date(row):
    #extract number and identifier (days or hours)
    
    advertised_number = int(re.findall('\d+', row['advertised_number_of_days_ago'])[0])
    
    if re.findall('hour', row['advertised_number_of_days_ago']):
        return row['query_date']
    elif re.findall('day', row['advertised_number_of_days_ago']) and advertised_number == 30:
        return row['query_date'] - pd.Timedelta(days=30) #jobs advertised 30+ days ago also defaulted to 30 days.
    elif re.findall('day', row['advertised_number_of_days_ago']) and advertised_number != 30:            
        return row['query_date'] - pd.Timedelta(days=advertised_number)
    else:
        return None

In [5]:
# apply to the dataframe
df_joblist['advertised_date'] = df_joblist.apply (lambda row: compute_advertised_date(row),axis=1)

## 4) Writing Back to Master

### dedup process

In [6]:
# if file does not exist write header 
indeed_job_board_listing = pd.read_csv('indeed_job_board_listing.csv', encoding='utf-8')
current_records = len(indeed_job_board_listing)
if not os.path.isfile('indeed_job_board_listing.csv'):
    df_joblist.to_csv('indeed_job_board_listing.csv',index = False, encoding='utf-8')
else: # else it exists so append without writing the header
#    df_joblist.to_csv('indeed_job_board_listing.csv',mode = 'a',header=False, index = False, encoding='utf-8')   
    indeed_job_board_listing = pd.concat((indeed_job_board_listing,df_joblist))
    indeed_job_board_listing.reset_index(inplace = True)
    del indeed_job_board_listing["index"]
    indeed_job_board_listing.sort_index(ascending = True, inplace = True)
    len(indeed_job_board_listing)
indeed_job_board_listing.to_csv('indeed_job_board_listing.csv',index = False, encoding='utf-8')

In [7]:
dedup_indeed_job_board_listing = pd.read_csv('indeed_job_board_listing.csv', encoding='utf-8')
dup_identifiers = ['summary', 'jobtitle', 'companyname', 'location']
# keep first because we want to retain the earliest posting of the job
dedup_indeed_job_board_listing = dedup_indeed_job_board_listing.drop_duplicates(keep = 'first',subset=dup_identifiers)

In [8]:
dedup_indeed_job_board_listing.to_csv('indeed_job_board_listing.csv',index = False, encoding='utf-8')

#### Rewrite file with deduped records 

In [9]:
# if file does not exist write header
dedup_indeed_job_board_listing.to_csv('indeed_job_board_listing.csv',index = False, encoding='utf-8')

### Process Summary

In [10]:
print "Duplicate records:",len(indeed_job_board_listing)-len(dedup_indeed_job_board_listing)
print "New records added:",len(dedup_indeed_job_board_listing)-current_records
print "Total Records:", len(dedup_indeed_job_board_listing)
print "Latest date is ",dedup_indeed_job_board_listing.sort_index().query_date.iloc[0]

Duplicate records: 29617
New records added: 10969
Total Records: 47054
Latest date is  10/08/2016
